In [4]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
word_to_idx = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 20)
lookup_tensor = torch.LongTensor([word_to_idx["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:

Columns 0 to 9 
 0.4414  0.6102 -2.1733 -0.6532 -0.8723  0.4362 -0.6678  1.1770  0.6404 -0.4654

Columns 10 to 19 
 0.6054  0.3471  0.8202  0.8289 -0.3998  1.5025 -0.0489 -1.0741  1.2778  0.0211
[torch.FloatTensor of size 1x20]



In [6]:
# Setting up word indices
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

print(trigrams[:3])

vocab = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocab)}

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [7]:
# Defining the n-gram model

class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.l1 = nn.Linear(context_size * embedding_dim, 128)
        self.l2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.l1(embeds))
        out = self.l2(out)
        log_probs = F.log_softmax(out, dim = 1)
        return log_probs

In [8]:
# Define the loss and optimizer
losses = []
loss_fn = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr = 1e-3)

In [14]:
# Train the model
for epoch in range(1000):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        context_idxs = [word_to_idx[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        model.zero_grad()
        
        log_probs = model(context_var)
        loss = loss_fn(log_probs, autograd.Variable(torch.LongTensor([word_to_idx[target]])))
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
    
print(losses)

[
 520.9530
[torch.FloatTensor of size 1]
, 
 518.4242
[torch.FloatTensor of size 1]
, 
 515.9129
[torch.FloatTensor of size 1]
, 
 513.4169
[torch.FloatTensor of size 1]
, 
 510.9366
[torch.FloatTensor of size 1]
, 
 508.4707
[torch.FloatTensor of size 1]
, 
 506.0168
[torch.FloatTensor of size 1]
, 
 503.5755
[torch.FloatTensor of size 1]
, 
 501.1470
[torch.FloatTensor of size 1]
, 
 498.7287
[torch.FloatTensor of size 1]
, 
 496.3202
[torch.FloatTensor of size 1]
, 
 493.9209
[torch.FloatTensor of size 1]
, 
 491.5294
[torch.FloatTensor of size 1]
, 
 489.1467
[torch.FloatTensor of size 1]
, 
 486.7708
[torch.FloatTensor of size 1]
, 
 484.4007
[torch.FloatTensor of size 1]
, 
 482.0362
[torch.FloatTensor of size 1]
, 
 479.6761
[torch.FloatTensor of size 1]
, 
 477.3195
[torch.FloatTensor of size 1]
, 
 474.9653
[torch.FloatTensor of size 1]
, 
 472.6154
[torch.FloatTensor of size 1]
, 
 470.2680
[torch.FloatTensor of size 1]
, 
 467.9241
[torch.FloatTensor of size 1]
, 
 465.5819

In [15]:
# Testing model
context = ['When', 'forty']
context_idxs = [word_to_idx[w] for w in context]
context_var = autograd.Variable(torch.LongTensor(context_idxs))
log_probs = model(context_var)

values, indices = log_probs.max(1)

print(values)
print(indices)
print(list(vocab)[indices.data[0]])

Variable containing:
1.00000e-02 *
 -2.7756
[torch.FloatTensor of size 1]

Variable containing:
 77
[torch.LongTensor of size 1]

winters


In [20]:
# Generate a sentence based on two input words
word1 = 'When'
word2 = 'forty'

print(word1, word2)
for i in range(10):
    context = [word1, word2]
    context_idx = [word_to_idx[w] for w in context]
    context_var = autograd.Variable(torch.LongTensor(context_idxs))
    
    log_probs = model(context_var)
    values, indices = log_probs.max(1)
    
    word1 = word2
    word2 = list(vocab)[indices.data[0]]
    print(word1, word2)

When forty
forty winters
winters winters
winters winters
winters winters
winters winters
winters winters
winters winters
winters winters
winters winters
winters winters
